In [1]:
def find_roi(data, segmentation, anat, basestructs, labelsdf, roi_names, num_echos):
    """
    Returns the roi which is showing significant pixel intensities compared to the baseline
    """
    zthr=-1.95
    roiDict=create_dict(roi_names, labelsdf, basestructs, num_echos)

    for k, roi_name in enumerate(roi_names):
        roi_areas=roiDict[roi_name].keys()
        zmap = np.zeros((len(roi_areas), np.shape(segmentation)[0], np.shape(segmentation)[1]))
        zmap_nii = np.zeros_like(segmentation)
        for j,roi in enumerate(roi_areas):
            img_slice = np.unique(np.where(segmentation==roi)[-1])[0]
            segmentation_sliced = segmentation[:,:,img_slice]
            anat_sliced = anat[:,:,img_slice]
            if num_echos>0:
                data_sliced = data[:,:,img_slice,:]
            else:
                data_sliced = data[:,:,img_slice]
                
            if num_echos>0:
                for echo in range(num_echos):
                    img=data_sliced[:,:,echo]

                    for index in np.array(list(zip(*np.where(segmentation_sliced==roi)))):
    #                     base_seg,struct,_,_=get_neigh_basestruct(index, basestructs, labelsdf, segmentation)
                        base_seg=anat_sliced[index[0], index[1]]
                        struct=labelsdf["Anatomical Regions"][labelsdf["Labels"]==base_seg]
                        if struct.any():
                            struct=struct.values[0]
                            if struct not in basestructs:
                                base_seg,struct,_,_=get_neigh_basestruct(index, basestructs, labelsdf, segmentation_sliced)
                            baseline=get_baseline_vals(base_seg, segmentation_sliced, data_sliced)        
                            imgz=img_zscore(img[index[0],index[1]],baseline[:,echo])

                            if imgz<zthr:
                                roiDict[roi_name][roi][struct][echo]['index'].append(index)
                                roiDict[roi_name][roi][struct][echo]['intensity'].append(img[index[0],index[1]])

                            #mask of the roi
#                             masks[k, j,echo,index[0],index[1]]=1

            else:
                echo=0
                img=data_sliced
                for index in np.array(list(zip(*np.where(segmentation_sliced==roi)))):
                    base_seg=anat_sliced[index[0], index[1]]
                    struct=labelsdf["Anatomical Regions"][labelsdf["Labels"]==base_seg]
                    if struct.any():
                        struct=struct.values[0]
                        if struct not in basestructs:
                            base_seg,struct,_,_=get_neigh_basestruct(index, basestructs, labelsdf, segmentation_sliced)
                        baseline=get_baseline_vals(base_seg, segmentation_sliced, data_sliced)        
                        imgz=img_zscore(img[index[0],index[1]],baseline)

                        if imgz<zthr:
                            roiDict[roi_name][roi][struct]['index'].append(index)
                            roiDict[roi_name][roi][struct]['intensity'].append(img[index[0],index[1]])
                            zmap[j, index[0], index[1]] = abs(imgz)*10
                            zmap_nii[index[0], index[1], img_slice] = abs(imgz)*10
                        #mask of the roi
#                         masks[k, j,index[0],index[1]]=1
    return roiDict, zmap, zmap_nii

In [ ]:
def get_neigh_basestruct(idx, basestructs, labelsdf, segmentation_mask):
    """
    idx: index to be checked which baseline is nearest to
    basestructs: list of struct names as possible baselines
    labelsdf: df including the segmentation names and values
    segmentation_mask: corresponding segmentation img slice
    """
    dist=[]
    neighidx=[]

    # Find the anatomical baseline structures existing in the given labels file.
    overlap_basestructures = list(set(basestructs) & set(labelsdf["Anatomical Regions"]))
    
    for base in overlap_basestructures:
        base_seg=labelsdf["Labels"][labelsdf["Anatomical Regions"]==base].values[0]
        baseidx=np.array(list(zip(*np.where(segmentation_mask==base_seg))))
        normarr=np.linalg.norm(baseidx-idx, axis=1)
        closestidx=np.argmin(normarr)

        dist.append(normarr[closestidx])
        neighidx.append(baseidx[closestidx])
        
    struct=basestructs[np.argmin(dist)]
    base_seg=labelsdf["Labels"][labelsdf["Anatomical Regions"]==struct].values[0]
    return base_seg, struct, neighidx, dist

In [ ]:
def get_centers(roiDict, num_echos=8):
    """
    finds centers of mass, and returns them as an numpy array
    """
    centers=np.zeros((len(roiDict.keys()), num_echos, 2))
    for i, roi in enumerate(roiDict.keys()):
        for echo in range(8):
            weightedSum=np.array([0,0])
            weightsSum=0
            for struct in roiDict[roi].keys():
                weights=np.reshape(((255-np.array(roiDict[roi][struct][echo]['intensity']))/255), (-1,1))
                indeces=np.array(roiDict[roi][struct][echo]['index'])
                if weights.any():
                    weightedSum=weightedSum+np.sum(indeces*weights, axis=0)
                    weightsSum=weightsSum+np.sum(weights)
            centers[i,echo,:]=weightedSum/weightsSum

    return centers

In [6]:
def get_effectsize(roiDict, num_echos=8):
    num_patterns=len(roiDict.keys())
    effectSizes=np.zeros((num_patterns, num_echos))
    for i, roi in enumerate(roiDict.keys()):
        for echo in range(num_echos):
            size=0
            for struct in roiDict[roi].keys():
                size=size+len(roiDict[roi][struct][echo]['index'])
#             for j,echo in enumerate(roiDict[roi][struct].keys()):
            effectSizes[i,echo]=size
                
    return np.transpose(effectSizes)

In [1]:
def create_dict(roi_names, labelsdf, basestructs, num_echos):
    data={}
    for roi_name in roi_names:
        roi_areas=labelsdf["Labels"][labelsdf["Anatomical Regions"].str.contains(roi_name)]
        # print(roi_areas)
        data[roi_name]={}
        for roi in roi_areas:
            data[roi_name][roi]={}
            
            for struct in basestructs:
                data[roi_name][roi][struct]={}
                if num_echos>0:
                    for echo in range(num_echos):
                        data[roi_name][roi][struct][echo]={"index": [],
                                                "intensity":[]}
                else:
                    data[roi_name][roi][struct]={"index": [],
                                        "intensity":[]}
    return data

In [1]:
def get_dist(centers, px_size):
    num_patterns=centers.shape[0]
    distIdx=[]
    for i in range(num_patterns-1):
        m=num_patterns
        j=i+1
        distIdx.append(m * i + j - ((i + 2) * (i + 1)) // 2)
    # dist(X[i,:], X[j,:]) = dists[m * i + j - ((i + 2) * (i + 1)) // 2], X: m-by-n
    
    dists=spatial.distance.pdist(centers, metric="euclidean")
    return dists[distIdx]*px_size

def get_dist_echos(centers):
    """
    centers: array; num_patterns-by-num_echos-by-2
    """
    num_patterns=centers.shape[0]
    if len(centers.shape)==3:
        num_echos=centers.shape[1]
    else:
        num_echos=1
    
    distIdx=[]
    for i in range(num_patterns-1):
        m=num_patterns
        j=i+1
        distIdx.append(m * i + j - ((i + 2) * (i + 1)) // 2)
    # dist(X[i,:], X[j,:]) = dists[m * i + j - ((i + 2) * (i + 1)) // 2], X: m-by-n

    distsEchos=np.empty((len(distIdx),num_echos))

    for echo in range(num_echos):
        dists=spatial.distance.pdist(centers[:,echo,:], metric="euclidean")
        distsEchos[:,echo]=dists[distIdx]*136

    return distsEchos

In [ ]:
def disk_roi(img,center, r):
    disk=skimage.morphology.disk(r)
    x,y=center
    mask=np.zeros_like(img)
    mask[x-r:x+r+1, y-r:y+r+1]=disk
    
    return mask

def sq_roi(img, center, pad=1):
    x,y=center
    mask=np.zeros_like(img)
    mask[x-pad:x+pad+1, y-pad:y+pad+1]=1
    
    return mask

In [3]:
def get_echo_vals(data, mask):
    num_echos=data.shape[-1]
    echoVals=np.zeros((num_echos, np.sum(mask).astype(int)))
    meanVals=np.zeros((num_echos,))
    stdVals=np.zeros((num_echos,))
    
    for echo in range(num_echos):
        img=data[:,:,echo]        
        
        echoVals[echo,:]=np.ndarray.flatten(img[mask>0])
        meanVals[echo]=np.mean(echoVals[echo,:])
        stdVals[echo]=np.std(echoVals[echo,:])
    
    return echoVals, meanVals, stdVals

In [ ]:
def argmax_roi_basestruct(mask, basestructs, labelsdf, segmentation):
    overlap=0
    argmax_value_overlap=0
    argmax_name_overlap=""
    for base in basestructs:
        base_seg_list=labelsdf["Labels"][labelsdf["Anatomical Regions"]==base]
        if base_seg_list.any():
            base_seg=base_seg_list.values[0]
            seg_mask=(segmentation==base_seg)
            if np.sum(mask*seg_mask)>overlap:
                overlap=np.sum(mask*seg_mask)
                argmax_value_overlap=base_seg
                argmax_name_overlap=base
                
    return argmax_value_overlap, argmax_name_overlap

# LEGACY CODE

In [6]:
def get_x(dictionary, roi, echo):
    idx_x=np.empty((1,))
    for struct in dictionary[roi].keys():
#         for echo in dictionary[roi][struct].keys():
        if dictionary[roi][struct][echo]['index']:
            newX=np.array(dictionary[roi][struct][echo]['index'])[:,1]
            idx_x=np.concatenate((idx_x, newX))
    return idx_x[1:]

In [ ]:
def pad_arr(arr, pad):
    arr=np.concatenate((np.linspace(arr[0]-pad, arr[0]-1, pad), arr, np.linspace(arr[-1]+1, arr[-1]+pad, pad)))
    return arr

In [2]:
## LEGACY FUNCTION ##
def get_endpoints(data, segments, centers, masks, echo, save_path="", filename=""):

    closePoints=np.empty((len(segments),2))
    farPoints=np.empty((len(segments),2))

    for i, seg in enumerate(segments):
        if masks[i, echo, :, :].any():
            center1=centers[i,echo,:]

            if i==0:
                center2=centers[i+1,echo,:]
                u21=(center2-center1)/np.linalg.norm(center2-center1)
                u12=u21
            elif i==len(segments)-1:
                center3=centers[i-1,echo,:]
                u12=(center1-center3)/np.linalg.norm(center3-center1)
                u21=u12
            else:
                center2=centers[i+1,echo,:]
                center3=centers[i-1,echo,:]
                u21=(center2-center1)/np.linalg.norm(center2-center1)
                u12=(center1-center3)/np.linalg.norm(center3-center1)
            
            alpha=1
            while True: 
                closeP=np.round(center1+alpha*u21)
                y,x=closeP.astype(int)
                if masks[i, echo, x, y]==1:
                    alpha=alpha+1
                else:
#                     closeP=closeP-u21
                    break
            alpha=1
            while True: 
                farP=np.round(center1-alpha*u12)
                y,x=farP.astype(int)
                if masks[i, echo, x, y]==1:
                    alpha=alpha+1
                else:
#                     farP=farP+u12
                    break

            closePoints[i,:]=closeP
            farPoints[i,:]=farP

        else:
            closePoints[i,:]=center1
            farPoints[i,:]=center1

        
    plt.figure()
    plt.imshow(data[:,:,img_slice,echo], cmap='gray')
    plt.scatter(weightedCenters[:,echo,0],weightedCenters[:,echo,1], c='r', s=0.1)
    plt.scatter(closePoints[:,0], closePoints[:,1],marker="*", c='b', s=0.01)
    plt.scatter(farPoints[:,0], farPoints[:,1],marker="*", c='m', s=0.01)
    if save_path and filename:
        filename="endpoints-echo"+str(echo)+"-electrode"+filename+".png"
        plt.savefig(save_path+filename, dpi=dpi)
        
    return farPoints, closePoints

In [ ]:
## LEGACY FUNCTION ## 
def get_centers_dict(roiDict, num_echos=8):
    """
    finds centers of mass and puts them into roi-dictionary
    """
    for roi in roiDict.keys():
        for echo in range(8):
            weightedSum=np.array([0,0])
            weightsSum=0
            for struct in roiDict[roi].keys():
                weights=np.reshape(((255-np.array(roiDict[roi][struct][echo]['intensity']))/255), (-1,1))
                indeces=np.array(roiDict[roi][struct][echo]['index'])
                if weights.any():
                    weightedSum=weightedSum+np.sum(indeces*weights, axis=0)
                    weightsSum=weightsSum+np.sum(weights)
            roiDict[roi]['center_echo'+str(echo)]=weightedSum/weightsSum
    return roiDict

In [ ]:
def get_c2c_dist_legacy(segment_lengths, intersegment_lengths):
    num_patterns=len(segment_lengths)
    c2c=np.zeros((num_patterns-1,))
    
    for i in range(num_patterns-1):
        c2c[i]=segment_lengths[i]/2 + intersegment_lengths[i] + segment_lengths[i+1]/2
    
    return c2c